# Set storage

You can change the storage location as many times you want using `set_storage` in the API or `lndb set --storage` on the CLI.

In [ ]:
from lndb import settings, load, init, login, set_storage
from lndb._clone import setup_local_test_postgres
from pathlib import Path

## Set storage for PostgreSQL instance

In [ ]:
pgurl = setup_local_test_postgres()

In [ ]:
init(storage="storage_1", db=pgurl)

Running 
```
!lndb set --storage storage_2
```
on the command line runs the following Python function:

In [ ]:
set_storage("storage_2")

In [ ]:
assert settings.instance.storage.root.as_posix() == f"{Path.cwd()}/storage_2"

Running 
```
lndb set --storage s3://lndb-setup-ci
```

on the command line runs the following:

In [ ]:
set_storage("s3://lndb-setup-ci")

In [ ]:
assert settings.instance.storage.is_cloud == True
assert str(settings.instance.storage.root) == "s3://lndb-setup-ci"

### Set storage not by owner

In [ ]:
login("testuser2@lamin.ai", password="goeoNJKE61ygbz1vhaCVynGERaRrlviPBVQsjkhz")

In [ ]:
assert set_storage("storage_3") == "only-owner-can-set-storage"

In [ ]:
!docker stop pgtest && docker rm pgtest

## Set storage fo SQLite instance

If the user tries to set the storage for an sqlite instance, an error message is raised:

In [ ]:
login("testuser1")

In [ ]:
load("mydata")

In [ ]:
assert set_storage("mydata_storage_2") == "set-storage-failed"